# JESS API Example
This notebook shows a simple example of running simulations and retrieving results using the JESS API as part of the ENSIMS Web Service platform. Documentation of the service and the API are under development. This demo can be used as a get started guide.

* [Get Connected](#first-bullet)
* [Authentication](#auth-bullet)
* [Upload & Run](#eplus-bullet)
* [Zip, Upload & Run](#jeplus-bullet)
* [Check Job Status](#status-bullet)
* [Retrieve Result File List](#list-bullet)
* [Retrieve a File](#retrieve-bullet)
* [Download All Results](#download-bullet)
* [View Results Online](#view-online-bullet)



## Get Connected <a class="anchor" id="first-bullet"></a>

Let's start with loading the necessary package and testing the connection to the server. The "[Requests: HTTP for Humans](http://python-requests.org "Go to Python Requests website")" is shown here, although other http packaages can be used too. 

After loading the Request package, we can run the Info command. Info is the simplest command and does not require authentication, so it works also if you copy and paste the URL to a browser, or [click here](https://api.ensims.com/jess_web/api/info "Try Info command in browser").

In [32]:
# Use requests (see https://requests.readthedocs.io/en/master/)
import requests

# API endpoints
ApiBase = 'https://api.ensims.com/'
JessApi = ApiBase + "jess_web/api/"
UserApi = ApiBase + 'users/api/'

# Test the connection
r = requests.get(JessApi + 'info')
r.json()

{'Title': 'JESS Web API',
 'Description': 'JESS online simulation services API provided by ENSIMS Ltd.',
 'Major': 1,
 'Minor': 0,
 'Revision': 0,
 'Release': 'beta',
 'Update': 1,
 'Notice': '(C) 2020, Energy Simulation Solutions Ltd. All rights reserved.'}

The request returns data in JSON format, which most of the JESS Web API uses. To see the data, use the 'json()' function. We can also access its fields like this:

In [2]:
r.json()['Title']

'JESS Web API'

## Authentication <a class="anchor" id="auth-bullet"></a>

First, we need to log on to the ENSIMS Web Service platform. If you haven't got an ENSIMS Web account, please go to [app.ensims.com](https://app.ensims.com/?user=register "Go to app.ensims.com") and register one. You must also request a new JESS account or link an existing one before you can perform the rest of the tasks on this page. See [this intro video](https://youtu.be/LQmx2BiET70 "Youtube video link") for the process.

Here, we just log on using the existing credential and then store the cookies. The Log on command is a POST request with appropriate headers and the user credentials in JSON format as the data body. Here is how you do it. **Remember to replace the email field with your own.**

In [33]:
# Get log in credential
import getpass
user_email = "demo1@jeplus.org" # <== Change this to your registered email
password = getpass.getpass("Password for " + user_email + ": ")

# Set header and body of the POST request
headers = {'Content-Type': 'application/json'}
body = {"email": user_email, "password": password}

# Send request
r = requests.post(UserApi + 'auth', headers=headers, json=body)

# Keep the cookies which contain the session token for secured requests
cookies = r.cookies

# Show data returned by JESS
r.json()

Password for demo1@jeplus.org:  ········


{'ok': True,
 'status': 'Logged in successfully!',
 'jwt': 'Session token in cookie',
 'user': 'Yi',
 'role': None,
 'email': 'demo1@jeplus.org'}

Now you are logged on and the session cookie is stored in the "cookies" variable.

## Upload & Run - EnergyPlus <a class="anchor" id="eplus-bullet"></a>

The quickest way to run simulations is by using the "upload and run" command (POST job/). This involves wrapping all the information about the simulation job to run, including the necessary files, in multipart form data. Here is an example.

In [34]:
import os

# detect the current working directory and print it
path = os.getcwd()
print ("The current working directory is %s" % path)

# upload a file to a particular folder. Becareful that the file name fields and the model/weather fields must match!
files = [
    ('file', ('5ZoneAirCooled-v93.idf', open('job_example\\5ZoneAirCooled-v93.idf', 'rb'), 'text/plain')),
    ('file', ('in.epw', open('job_example\\in.epw', 'rb'), 'text/plain')),
    ('title', 'Python test case'),
    ('desc', 'This is test submission made from the API example for Python'),
    ('split', 'FALSE')
]

# POST with files
r = requests.post(JessApi + 'job', files=files, cookies=cookies)

# Get job_id. This id number will be needed for querying and retrieving the job data
job_id = r.json()['data']

# Show the returned status
r.json()

The current working directory is C:\dev\python\JESS-Web-API


{'ok': True, 'status': 'Job 385114 is created', 'data': 385114}

If job submission is successful, JESS returns an acknowledgement with the ok status set to True, and the job id in the 'data' field. Take a note of the **job id** as you will need to access the job status and the simulation results later.

## Zip, Upload & Run - jEPlus Project <a class="anchor" id="jeplus-bullet"></a>

To upload more files, especially if the folder structure need to be maintained, it may be best to zip the files and folders before upload & run. This exmaple shows how to do it with a jEPlus project folder.

In [38]:
from shutil import make_archive

# detect the current working directory and print it
path = os.getcwd()
print ("The current working directory is %s" % path)

# zip all files and sub-folders in the jeplus_example folder into files.zip
toUpload = make_archive('files', 'zip', root_dir=os.path.join(path, 'jeplus_example'))
print (toUpload + " is prepared and to be uploaded")

# upload the zip file and set the jEPlus project options
files = [
    ('file', ('files.zip', open(toUpload, 'rb'), 'application/zip')),
    ('type', 'JEP'),      # Important - JEP for jeplus projects and EP for E+ models
    ('title', 'jEPlus project example'),
    ('desc', 'This is test submission made from the API example for Python'),
    ('subset', 'LHS'),    # Or ('subset', 'LIST_FILE'), ('cases', 'jobs.txt')
    ('cases', '10')
]

# POST with files
r = requests.post(JessApi + 'job', files=files, cookies=cookies)

# Get job_id. This id number will be needed for querying and retrieving the job data
# job_id = r.json()['data']

# Show the returned status
r.json()



The current working directory is C:\dev\python\JESS-Web-API
C:\dev\python\JESS-Web-API\files.zip is prepared and to be uploaded


{'ok': True, 'status': 'Job 385117 is created', 'data': 385117}

## List Jobs <a class="anchor" id="jobs-bullet"></a>


You can list the jobs available on the server using the following operation. Note the filter showing only the jobs that are submitted today and have finished simulation.

In [39]:
# !! This part is not working !!
from datetime import date

today = date.today()

# YYYY-mm-dd
dstr = today.strftime("%Y-%m-%d")
print("Retrieve jobs list on " + dstr)

# GET the list of jobs fit the given criteria
filter = {"since": dstr, "till": dstr, "status": "FINISHED"}

r = requests.get(JessApi + 'jobs', headers=headers, json=filter, cookies=cookies)

r.json()

Retrieve jobs list on 2021-07-06


{'ok': True,
 'status': 'yiz0 jobs available',
 'data': [{'job_Desc': 'This is test submission made from the API example for Python',
   'job_SN': 385099,
   'job_Subset': 'ALL',
   'submissionSize': 1711719,
   'job_Args': '',
   'received': 1625569133430,
   'njobs': 1,
   'cputime': 6,
   'storageSize': 15072020,
   'resultSize': 0,
   'job_Name': 'Python test case',
   'progress': '1, 0, 0, 0, 0, 0, 1',
   'accepted': None,
   'retrieved': None,
   'cancelled': None,
   'simulated': 1625569157970,
   'prepared': 1625569137515,
   'collected': 1625569158028,
   'id': 1189,
   'status': 'FINISHED',
   'username': 'yiz0',
   'job_Type': 'EP_IDF',
   'message': 'Job finished. Results collected into tables.'},
  {'job_Desc': 'This is test submission made from the API example for Python',
   'job_SN': 385114,
   'job_Subset': 'ALL',
   'submissionSize': 1711719,
   'job_Args': '',
   'received': 1625587116738,
   'njobs': 1,
   'cputime': 6,
   'storageSize': 15072020,
   'resultSize': 0

## Check Job Status <a class="anchor" id="status-bullet"></a>

To check the status of a particular job, use the following operation:

In [5]:
# GET job status with job_id
r = requests.get(JessApi + 'job/status/' + str(job_id), cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': '385099 status available',
 'data': {'progress': '1, 0, 0, 0, 0, 0, 1',
  'completion_Time': 1625569158028,
  'job_ID': 385099,
  'cancel_Flag': False,
  'last_Update': 1625569158038,
  'status_Info': 'Job finished. Results collected into tables.',
  'id': 385043,
  'status': 'FINISHED'}}

## Retrieve Result File List <a class="anchor" id="list-bullet"></a>

When simulation has finished, you can retrieve the list of result files using the following operation. The returned data are formated for rendering with JSTree, hence it may be too much to read.

In [6]:
# GET job status with job_id
r = requests.get(JessApi + 'job/dir/' + str(job_id), cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': '385099 status available',
 'data': {'state': {'disabled': False, 'openned': False, 'selected': False},
  'text': '385099 [*]',
  'data': {'fullPath': '/', 'folder': True, 'lastModified': 0, 'size': 0},
  'type': 'root',
  'id': None,
  'a_attr': {'title': '/ total size 0 B'},
  'li_attr': {},
  'children': [{'state': {'disabled': False,
     'openned': False,
     'selected': False},
    'text': 'console.log (5.42 kB)',
    'data': {'fullPath': '/console.log',
     'folder': False,
     'lastModified': 1625569146000,
     'size': 5416},
    'type': 'log',
    'id': None,
    'a_attr': {'title': '/console.log last modfied at 2021-07-06 10:59 UTC'},
    'li_attr': {},
    'children': []},
   {'state': {'disabled': False, 'openned': False, 'selected': False},
    'text': 'eplusout.bnd (62.31 kB)',
    'data': {'fullPath': '/eplusout.bnd',
     'folder': False,
     'lastModified': 1625569146000,
     'size': 62315},
    'type': 'bnd',
    'id': None,
    'a_attr':

## Retrieve a File <a class="anchor" id="retrieve-bullet"></a>

You can retrieve any result file in the text form from the server like this:

In [7]:
# GET specific job output with job_id and file name
r = requests.get(JessApi + 'job/file/' + str(job_id) + "/eplusout.err", cookies=cookies)

# Show the returned status
print(r.text)

Program Version,EnergyPlus, Version 9.3.0-baff08990c, YMD=2021.07.06 11:59,
   ************* Beginning Zone Sizing Calculations
   ** Warning ** Weather file location will be used rather than entered (IDF) Location object.
   **   ~~~   ** ..Location object=CHICAGO_IL_USA TMY2-94846
   **   ~~~   ** ..Weather File Location=LONDON/GATWICK - GBR IWEC Data WMO#=037760
   **   ~~~   ** ..due to location differences, Latitude difference=[9.37] degrees, Longitude difference=[87.57] degrees.
   **   ~~~   ** ..Time Zone difference=[6.0] hour(s), Elevation difference=[67.37] percent, [128.00] meters.
   ************* Beginning System Sizing Calculations
   ************* Beginning Plant Sizing Calculations
   ************* Testing Individual Branch Integrity
   ************* All Branches passed integrity testing
   ************* Testing Individual Supply Air Path Integrity
   ************* All Supply Air Paths passed integrity testing
   ************* Testing Individual Return Air Path Integrit

## Download All Files <a class="anchor" id="download-bullet"></a>

Or download all files of the job as a zip package, and write it to the local storage.

In [8]:
# Download all job output in as a zip file
r = requests.get(JessApi + 'job/file/' + str(job_id), cookies=cookies)

# Create the download/ folder to save the zip file
dl_path = "download\\"
if not os.path.exists(dl_path):
    try:
        os.mkdir(dl_path)
    except OSError:
        print ("Creation of the directory %s failed" % dl_path)
    else:
        print ("Successfully created the directory %s " % dl_path)

# Save the contents of the downloaded file
bytes = open(dl_path + str(job_id) + '.zip', 'wb').write(r.content)

# Show the returned status
print(str(bytes) + ' bytes written to ' + dl_path + str(job_id) + '.zip')

3131277 bytes written to download\385099.zip


## View Result Files Online <a class="anchor" id="view-online-bullet"></a>

You can generate links the viewers of particular output files and view them online.

In [40]:
print ("DXF model of " + str(job_id) + ": https://app.ensims.com/3d/?dxf=/" + str(job_id) + "/eplusout.dxf")
print ("IDF model of " + str(job_id) + ": https://app.ensims.com/idf/?ref=/" + str(job_id) + "/in.idf")
print ("ESO output of " + str(job_id) + ": https://app.ensims.com/eso/?ref=/" + str(job_id) + "/eplusout.eso")
print ("Html output of " + str(job_id) + ": https://app.ensims.com/jess_web/api/job/text/" + str(job_id) + "/eplustbl.htm")

DXF model of 385114: https://app.ensims.com/3d/?dxf=/385114/eplusout.dxf
IDF model of 385114: https://app.ensims.com/idf/?ref=/385114/in.idf
ESO output of 385114: https://app.ensims.com/eso/?ref=/385114/eplusout.eso
Html output of 385114: https://app.ensims.com/jess_web/api/job/text/385114/eplustbl.htm


## End of Demo

These are the basic operations for running simulations on JESS and retrieving the results. Check out other examples for more use scenarios and functions.
